In [1]:
from langchain.agents import Tool
from SPARQLWrapper import SPARQLWrapper, JSON

def query_virtuoso_kg(sparql_query: str) -> str:
    sparql = SPARQLWrapper("http://192.168.133.39:8890/sparql/")  # 你自己的 SPARQL endpoint
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return str(results["results"]["bindings"])

# 註冊為工具
sparql_tool = Tool(
    name="VirtuosoKGQuery",
    func=query_virtuoso_kg,
    description="查詢本地知識圖譜的 SPARQL 工具，使用 RDF 知識"
)


In [2]:
from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
#os.environ["GROQ_API_KEY"] = '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'
#API#2
#s.environ["GROQ_API_KEY"] = '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'
#API#3
os.environ["GROQ_API_KEY"] = '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'
#API#4
#os.environ["GROQ_API_KEY"] = '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'
#API#5
#os.environ["GROQ_API_KEY"] = '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'

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],

    model_name="llama-3.3-70b-versatile",       

    temperature=0.0,
  #  max_tokens=1000,
)

In [13]:
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

#llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=[sparql_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
)

response = agent.run("請查詢 '蔣中正' 的職業")


NameError: name 'sparql_tool' is not defined

In [ ]:
def query_virtuoso_kg(sparql_query: str) -> str:
    sparql = SPARQLWrapper("http://192.168.133.39:8890/sparql/")  # 你自己的 SPARQL endpoint
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return str(results["results"]["bindings"])

# 註冊為工具
sparql_tool = Tool(
    name="VirtuosoKGQuery",
    func=query_virtuoso_kg,
    description="查詢本地知識圖譜的 SPARQL 工具，使用 RDF 知識"
)


In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.tools import Tool

# 你的 local_query_sparql 函式
def local_query_sparql(sparql: str) -> dict:
    sparql = SPARQLWrapper("http://192.168.133.39:8890/sparql/")  # 你自己的 SPARQL endpoint
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return str(results["results"]["bindings"])

# 1. 包成 Tool
sparql_query_tool = Tool.from_function(
    name="LocalSPARQLQuery",
    description="查詢本地 SPARQL endpoint。輸入必須是一個 SPARQL 查詢字串。",
    func=local_query_sparql,
)

# 2. 設定 LLM
#llm = ChatOpenAI(model="gpt-3.5-turbo")


In [6]:
from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
#os.environ["GROQ_API_KEY"] = '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'
#API#2
#s.environ["GROQ_API_KEY"] = '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'
#API#3
os.environ["GROQ_API_KEY"] = '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'
#API#4
#os.environ["GROQ_API_KEY"] = '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'
#API#5
#os.environ["GROQ_API_KEY"] = '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'

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],

    model_name="llama-3.3-70b-versatile",       

    temperature=0.0,
  #  max_tokens=1000,
)

In [12]:

# 3. 建立 Prompt（這很重要！）
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是專門幫助使用者查詢知識圖譜的助手，可以使用 SPARQL 查詢工具。請適時呼叫工具查詢。"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 4. 建立 Agent
agent = create_openai_functions_agent(
    llm=llm,
    tools=[sparql_query_tool],
    prompt=prompt,
)

# 5. 建立 AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=[sparql_query_tool])

# 6. 測試執行
response = agent_executor.invoke({
    "input": "請查詢包含蔣中正的資料"
})

print(response)

{'input': '請查詢包含蔣中正的資料', 'output': ''}
